# Lecture 7 👻

In [1]:
#Import statements
import hashlib as hasher
import random

def hashbits(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    hashbytes = hash_obj.digest()
    return ''.join(f'{x:08b}' for x in hashbytes)

def hash(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    return hash_obj.hexdigest()

def numberOfInitZeros(hashStr):
  count = 0
  for i in range (0 , len(hashStr)):
    if hashStr[i] == '1':
      break
    count += 1
  return count

class Block:
    def __init__(self, data, creator=None, previous=None, nonce=0):
        self.data = data
        if previous is None:
            self.previous = None
            self.previous_hash = ""
            self.creator = Miner(0 , "0")
            self.height = 0
        else:
            self.previous = previous
            self.previous_hash = previous.hash
            self.creator = creator
            self.height = previous.height+1
        self.nonce = nonce
        self.hash = self.hash_block()
        #this below is new
        self.children = []

    def hash_block(self):
        return hashbits(self.data+ self.creator.name + self.previous_hash + str(self.nonce))

    def print(self):
      print(self.data + " "+ self.creator.name + " " + str(self.height))

class Miner:
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = random.randint(0,100000)
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.lastBlock()
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        self.blockchain.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

class SelfishMiner(Miner):
  def __init__(self, miningPower, name, blockchain=None, power=0):
    super().__init__(miningPower, name, blockchain, power)
    self.privateBlocks = []

class AMiner(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    super().__init__(miningPower, name, blockchain)
    self.lastBlock = self.blockchain.chain[0]


def attackerHasControl(blockchain, attacker):
  if blockchain.longestChain().creator == attacker:
    return True
  return False


In [2]:
####### Drawing blockchain, not important
from IPython.display import HTML, display
from IPython.html.widgets.interaction import interact

def maxHeight(parent):
  if len(parent.children) == 0:
    return parent.height 
  max = 0
  for child in parent.children:
    m = maxHeight(child)
    if m> max:
      max = m
  return max
  

def drawBlockchain(parent, level, html, parentLevel, childN = 0, total = 0):
  color = "#AEF751"
  if parentLevel!=-1:
    color = "#7EDBF6"
    if type(parent.creator) is AMiner:
      color = "#F58A70"
    elif type(parent.creator) is SelfishMiner:
      color = "#F59AEE"
  parent.children.sort(key=lambda x: (maxHeight(x)), reverse=True)
  xx = childN
  level += childN
  html += '<g>'
  html += '<rect x="'+str(30+ 100*parent.height)+'" y="'+str(30+ 100*level)+'" width="60" height="60" stroke="black" stroke-width="1" fill="'+color+'" />'
  html += '<text x="'+str((60+ 100*parent.height))+'" y="'+str((60+ 100*level))+'" dominant-baseline="middle" text-anchor="middle" font-family="Verdana" font-size="10" font-weight="bold" fill="black">'+str(parent.data)+'</text>'
  if parentLevel != -1:
    if (parent.previous.children.index(parent)) == 0:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(95+ 100*parent.previous.height)+'" y2="'+str(60+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
    else:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(65+ 100*parent.previous.height)+'" y2="'+str(95+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
  html += '</g>'
  l = level
  childN = 0
  for child in parent.children:
    html,n, t = drawBlockchain(child, l, html, level, childN, total)
    if n > 0:
      childN += n
    if t > 0:
      total += t
    l = l+1
  return html, childN+ len(parent.children)-1, total+ len(parent.children)-1


def show(bc):
  htmll = ""
  html = ""
  htmll, n, t = drawBlockchain(bc.chain[0], 0, html, -1)
  html = '<svg height="'+str(115*(n+1))+'" width="'+str(130*maxHeight(bc.chain[0]))+'">'
  html += '<defs><marker id="markerArrow" markerWidth="10" markerHeight="10" refX="2" refY="6" orient="auto"><path d="M2,2 L2,11 L10,6 L2,2" style="fill: #000000;" /> </marker> </defs>'
  html += htmll
  html += '</svg>'
  display(HTML(html))

/usr/local/lib/python3.9/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


## Exercise 1

### 👻 GHOST Protocol 👻

One way to increase throughout along with security is to use GHOST protocol instead of longest chain. In this protocol, miners choose the heaviest subtree (subtree with most blocks) instead of the longest chain.

- Below there is a stub for blockchain. Each blockchain has a property name *protocol*. Based on this property, miners decide whether to follow the longest chain, or the heaviest.
- A property name *children* is added to the above stub for *Block*, which is a list containing all blocks that branch from this block. The add function in the blockchain is changed to also add the new block to the children of its previous block.
- Implement *numberOfBlocksInSubtree* function that returns the total number of blocks in a subtree with the help of the list of chidlren. 
- With the helo of *numberOfBlocksInSubtree*, implement the function *heaviestChain* to return the latest node of the heaviest chain.
- Run the code and compare the results of "heaviest" and "longest" protocols. 
- What is the difference of 'forks' in these two protocols?

In [ ]:
class Blockchain:
    def __init__(self, genesis_data, difficulty, protocol="longest"):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0
        self.protocol = protocol

    def lastBlock(self):
      if self.protocol == "longest":
        return self.longestChain()
      else:
        return self.heaviestChain(self.chain[0])
    
    def longestChain(self): 
      max = self.chain[0]
      for block in self.chain:
        if block.height > max.height:
          max = block
      return max

    def heaviestChain(self, root):
      #add this function

    def numberOfBlocksInSubtree(self, block):
      #add this function

    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        
    def print(self):
      for block in self.chain:
        block.print()
        print("________")


bc = Blockchain("0" , 6, "longest")
miners = []
for i in range(1,11):
  m = Miner(4 ,"m"+str(i), bc)
  miners.append(m)

while bc.size < 20:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()


bc2 = Blockchain("0" , 6, "ghost")
miners = []
for i in range(1,11):
  m = Miner(4 ,"m"+str(i), bc2)
  miners.append(m)

while bc2.size < 20:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()

In [10]:
#Solution
class Blockchain:
    def __init__(self, genesis_data, difficulty, protocol="longest"):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0
        self.protocol = protocol

    def lastBlock(self):
      if self.protocol == "longest":
        return self.longestChain()
      else:
        return self.heaviestChain(self.chain[0])
    
    def longestChain(self): 
      max = self.chain[0]
      for block in self.chain:
        if block.height > max.height:
          max = block
      return max

    def heaviestChain(self, root):
      if len(root.children) == 0:
        return root
      else:
        max = root.children[0]
        maxChilds = self.numberOfBlocksInSubtree(max)
        for block in root.children:
          tmp = self.numberOfBlocksInSubtree(block)
          if tmp > maxChilds:
            max = block
            maxChilds = tmp
        return self.heaviestChain(max)

    def numberOfBlocksInSubtree(self, block):
      if len(block.children) == 0:
        return 1
      else:
        sum = 0
        for b in block.children:
          sum += self.numberOfBlocksInSubtree(b)
        return 1 + sum

    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        
    def print(self):
      for block in self.chain:
        block.print()
        print("________")


bc = Blockchain("0" , 6, "longest")
miners = []
for i in range(1,11):
  m = Miner(4 ,"m"+str(i), bc)
  miners.append(m)

while bc.size < 20:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()


bc2 = Blockchain("0" , 6, "ghost")
miners = []
for i in range(1,11):
  m = Miner(4 ,"m"+str(i), bc2)
  miners.append(m)

while bc2.size < 20:
  for m in miners:
    m.PoWSolver()
  for m in miners:
    m.UpdateLast()


In [ ]:
show(bc)

In [ ]:
show(bc2)